In [24]:
import os  
import random  

In [25]:
trainval_percent = 0.7    # 訓練加驗證集佔全部資料集比重  
train_percent = 0.8       # 訓練集佔訓練加驗證集比重

xmlfilepath = 'D:/Project/Python/edgeAI/dataset/VOC2007/Annotations'   # xml檔案位置
txtsavepath = 'D:/Project/Python/edgeAI/dataset/VOC2007/ImageSets/Main'# txt檔案位置
total_xml = os.listdir(xmlfilepath)  

num = len(total_xml)  
list = range(num)  
tv = int(num*trainval_percent)  
tr = int(tv*train_percent)  
trainval = random.sample(list,tv)  
train = random.sample(trainval,tr)  

ftrainval = open(txtsavepath+'/trainval.txt', 'w')  
ftest = open(txtsavepath+'/test.txt', 'w')  
ftrain = open(txtsavepath+'/train.txt', 'w')  
fval = open(txtsavepath+'/val.txt', 'w')  

for i in list:  
    name = total_xml[i][:-4]+'\n'  
    if i in trainval:  
        ftrainval.write(name)  
        if i in train:  
            ftrain.write(name)  
        else:  
            fval.write(name)  
    else:  
        ftest.write(name)  

ftrainval.close()  
ftrain.close()  
fval.close()  
ftest.close()
print('已設置完成')

已設置完成


In [26]:
import os
from shutil import copyfile

In [28]:
# 根據tarin.txt和 test.txt將資料集分為標準資料集
train_text_path = 'D:/Project/Python/edgeAI/dataset/VOC2007/ImageSets/Main/train.txt'
test_text_path = 'D:/Project/Python/edgeAI/dataset/VOC2007/ImageSets/Main/test.txt'
# 圖片存放地址
image_path = 'D:/Project/Python/edgeAI/dataset/VOC2007/JPEGImages'
# xml檔案存放地址
xml_path = 'D:/Project/Python/edgeAI/dataset/VOC2007/Annotations'

# 輸出的目錄
outdir = 'D:/Project/Python/edgeAI/dataset/VOC2007'
# 建立各級資料夾
test_xml_out = os.path.join(outdir,'test/Annotations')
os.makedirs(test_xml_out)
os.makedirs(os.path.join(outdir,'test/ImageSets/Main'))
test_img_out = os.path.join(outdir,'test/JPEGImages')
os.makedirs(test_img_out)


train_xml_out = os.path.join(outdir,'train/Annotations')
os.makedirs(train_xml_out)
os.makedirs(os.path.join(outdir,'train/ImageSets/Main'))
train_img_out = os.path.join(outdir,'train/JPEGImages')
os.makedirs(train_img_out)


with open(train_text_path) as f:
    lines = f.readlines()
    for i in lines:
        img_save_path = os.path.join(train_img_out,i.rstrip('\n')+'.jpg')
        xml_save_path = os.path.join(train_xml_out, i.rstrip('\n') + '.xml')
        copyfile(os.path.join(image_path,i.rstrip('\n')+'.jpg'),img_save_path)
        copyfile(os.path.join(xml_path, i.rstrip('\n') + '.xml'), xml_save_path)
        print(i)
with open(test_text_path) as f:
    lines = f.readlines()
    for i in lines:
        img_save_path = os.path.join(test_img_out, i.rstrip('\n') + '.jpg')
        xml_save_path = os.path.join(test_xml_out, i.rstrip('\n') + '.xml')
        copyfile(os.path.join(image_path, i.rstrip('\n') + '.jpg'), img_save_path)
        copyfile(os.path.join(xml_path, i.rstrip('\n') + '.xml'), xml_save_path)
        print(i)

000001

000004

000005

000006

000011

000013

000014

000015

000016

000019

000020

000021

000032

000034

000037

000038

000043

000044

000045

000046

000049

000050

000051

000053

000056

000057

000059

000060

000061

000062

000063

000065

000066

000067

000068

000070

000071

000075

000076

000081

000086

000088

000089

000091

000092

000094

000095

000098

000099

000101

000102

000104

000105

000106

000107

000111

000112

000113

000115

000118

000119

000120

000121

000122

000123

000126

000127

000130

000135

000137

000138

000139

000140

000141

000142

000146

000148

000149

000150

000152

000153

000156

000157

000158

000159

000162

000163

000165

000166

000167

000171

000172

000173

000174

000177

000178

000179

000180

000182

000183

000184

000185

000188

000189

000193

000194

000195

000196

000198

000203

000205

000206

000207

000208

000209

000212

000213

000214

000216

000217

000218

000220

000221

000223

000224



In [22]:
import os
import argparse
import xml.etree.ElementTree as ET

In [29]:
def convert_voc_annotation(data_path, data_type, anno_path, use_difficult_bbox=True):

    # 定義狗的姿勢類別
    classes = ["sit", "lie", "stand"]
    
    img_inds_file = os.path.join(data_path, 'ImageSets', 'Main', data_type + '.txt')
    
    with open(img_inds_file, 'r') as f:
        txt = f.readlines()
        image_inds = [line.strip() for line in txt]

    with open(anno_path, 'a') as f:
        for image_ind in image_inds:
            image_path = os.path.join(data_path, 'JPEGImages', image_ind + '.jpg')
            annotation = image_path
            label_path = os.path.join(data_path, 'Annotations', image_ind + '.xml')
            root = ET.parse(label_path).getroot()
            objects = root.findall('object')
            for obj in objects:
                difficult = obj.find('difficult').text.strip()
                if (not use_difficult_bbox) and(int(difficult) == 1):
                    continue
                bbox = obj.find('bndbox')
                class_ind = classes.index(obj.find('name').text.lower().strip())
                xmin = bbox.find('xmin').text.strip()
                xmax = bbox.find('xmax').text.strip()
                ymin = bbox.find('ymin').text.strip()
                ymax = bbox.find('ymax').text.strip()
                annotation += ' ' + ','.join([xmin, ymin, xmax, ymax, str(class_ind)])
            print(annotation)
            f.write(annotation + "\n")
    return len(image_inds)


if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    # 標註檔路徑
    parser.add_argument("--data_path", default="D:/Project/Python/edgeAI/dataset/VOC2007")
    # train.txt的路徑
    parser.add_argument("--train_annotation", default="D:/Project/Python/edgeAI/dataset/VOC2007/train/ImageSets/Main/train.txt")
    # test.txt的路徑
    parser.add_argument("--test_annotation",  default="D:/Project/Python/edgeAI/dataset/VOC2007/test/ImageSets/Main/test.txt")
    flags = parser.parse_args(args=[])

    if os.path.exists(flags.train_annotation):os.remove(flags.train_annotation)
    if os.path.exists(flags.test_annotation):os.remove(flags.test_annotation)

    # 訓練集和測試集的路徑
    num1 = convert_voc_annotation(os.path.join(flags.data_path, 'D:/Project/Python/edgeAI/dataset/VOC2007'), 'train', flags.train_annotation, False)
    num3 = convert_voc_annotation(os.path.join(flags.data_path, 'D:/Project/Python/edgeAI/dataset/VOC2007'),  'test', flags.test_annotation, False)
  
    print('=> 訓練集的影像數量: %d\t 測試集的影像數量:%d' %(num1 , num3))

D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\000001.jpg 975,481,1169,602,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\000004.jpg 975,491,1160,606,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\000005.jpg 1291,479,1481,576,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\000006.jpg 970,493,1179,590,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\000011.jpg 1290,490,1473,578,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\000013.jpg 1302,493,1478,575,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\000014.jpg 988,488,1164,596,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\000015.jpg 1293,470,1473,570,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\000016.jpg 1294,476,1475,575,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\000019.jpg 1281,479,1479,570,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\000020.jpg 978,476,1169,596,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\000021.jpg 979,484,1157,597,1
D:/Project

D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001340.jpg 90,485,322,722,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001341.jpg 1178,464,1284,602,0 272,529,484,711,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001343.jpg 610,443,725,578,0 1487,572,1707,781,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001345.jpg 1222,459,1338,572,0 140,562,397,728,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001347.jpg 1199,362,1313,525,0 213,718,452,903,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001350.jpg 1208,435,1320,565,0 190,576,423,749,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001354.jpg 1193,444,1332,585,0 216,567,457,741,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001355.jpg 1208,465,1361,573,0 158,567,410,740,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001357.jpg 28,670,273,872,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001358.jpg 1697,438,1905,673,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImage

D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001398.jpg 714,626,782,718,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001501.jpg 1335,444,1457,546,0 476,614,701,734,1
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001508.jpg 1538,485,1729,812,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001509.jpg 1467,775,1649,1031,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001510.jpg 1472,797,1628,1032,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001518.jpg 149,582,351,781,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001520.jpg 131,562,346,782,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001525.jpg 114,555,349,773,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001526.jpg 119,555,341,778,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001532.jpg 117,564,355,775,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001533.jpg 129,575,351,779,0
D:/Project/Python/edgeAI/dataset/VOC2007\JPEGImages\001534.jpg 119,570,355,779,0
